In [ ]:
import requests
import pandas as pd
import numpy as np
!pip install geocoder
import geocoder

In [46]:
geo_cache = {}

In [60]:

def get_lat_lng(x):
    print('\n'+x)
    if x in geo_cache:
        res = geo_cache[x]
    g = geocoder.google(x)
    if g.latlng:
        geo_cache[x] = g.latlng
        res = g.latlng
    else:
        res = [pd.np.nan, pd.np.nan]
    print(res)
    return res

# print('bk', get_lat_lng('Brooklyn, NY'))
# print(get_lat_lng('Brownstone Bagel, Brooklyn, NY'))
# print(get_lat_lng('5th Avenue and Union Ave, Brooklyn, NY'))
# print(get_lat_lng('5t2323f Union afewfAve, Bro43oklyn, NY'))
# OSHA examples
# print(get_lat_lng('Tower Building Services Inc., New York, NY 10021'))
# print(get_lat_lng("FJC Security Services, New York, NY 10014"))




### Crime Deaths

In [48]:
# TODO need to do paging, this just squeaks under the 50k limit XXX 
crime_deaths_url = "https://data.cityofnewyork.us/resource/n98d-maqp.json?ofns_desc=MURDER%20%26%20NON-NEGL.%20MANSLAUGHTER&$limit=50000"
r = requests.get(crime_deaths_url)
crime_deaths_json = r.json()

In [49]:
print(len(crime_deaths_json))
# print(crime_deaths_json[0])
crime_deaths = pd.DataFrame.from_records(crime_deaths_json)
# crime_deaths

4909


In [50]:
crime_deaths.drop(['ofns_desc'], axis = 1, inplace = True)

In [51]:
crime_deaths["kind"] = "c"
crime_deaths
crime_deaths.columns = ['lat', 'lng', 'date', 'kind']

In [52]:
print(crime_deaths.isnull().sum())
# crime_deaths
# returns 2 crime deaths we don't have lat lng for, but they lack any location info, nothing to geocode

lat     2
lng     2
date    0
kind    0
dtype: int64


### Motor vehicle related deaths

In [53]:
# https://data.cityofnewyork.us/resource/9tve-dvm5.json
vehicle_deaths_url = "https://data.cityofnewyork.us/resource/arr5-wtax.json?&$limit=50000"
r = requests.get(vehicle_deaths_url)
vehicle_deaths_json = r.json()
vehicle_deaths = pd.DataFrame.from_records(vehicle_deaths_json)

In [54]:
# how many have missing data
# vehicle_deaths.isnull().sum()
# show those missing lat
# vehicle_deaths[vehicle_deaths.isnull()['latitude']]

In [78]:
# specific to motor vehicle data / rows
def maybe_get_location(row):
    borough = row.borough if row.borough is not pd.np.NaN else ''
    if row.latitude is not pd.np.NaN :
        return [row.latitude, row.longitude]
    elif row.cross_street_name is not pd.np.NaN:
        return get_lat_lng(row.cross_street_name+' '+borough+ ' New York')
    elif row.off_street_name is not pd.np.NaN:
        return get_lat_lng(row.off_street_name+' '+borough+ ' New York')
    elif row.on_street_name is not pd.np.NaN:
        return get_lat_lng(row.on_street_name+' '+borough+ ' New York')
    else:
        # print('no dice for ', row)
        return [pd.np.nan, pd.np.nan]

    
# https://stackoverflow.com/a/26887820/83859
# df.apply (lambda row: label_race (row),axis=1) # just to see result


In [ ]:
# pass it the row
mvd_locs = vehicle_deaths.apply (lambda row: maybe_get_location (row),axis=1)

In [75]:
# turn the series of lists [lat,lng] into it's own frame, i.e. two series
mvd_loc_df = mvd_locs.apply(pd.Series)

In [ ]:
# vehicle_deaths[vehicle_deaths.isnull()['latitude']]
mvd_loc_df.isnull().sum()

In [80]:
# add locations from our other table
vehicle_deaths['latitude'] = mvd_loc_df[0]
vehicle_deaths['longitude'] = mvd_loc_df[1]

In [87]:
# vehicle_deaths.columns
print(vehicle_deaths.isnull().sum())
# vehicle_deaths[vehicle_deaths.isnull()['latitude']]

date     0
lat     72
lng     72
kind     0
dtype: int64


In [84]:
vehicle_deaths.drop(['borough',  'cross_street_name', 'location', 'off_street_name', 'on_street_name', 'zip_code'], axis = 1, inplace = True)
vehicle_deaths.columns = ['date', 'lat','lng' ] # 'num'
vehicle_deaths['kind']= 'v'


### Deaths by Police


In [96]:
by_police = pd.concat([pd.read_csv('https://raw.githubusercontent.com/flother/thecounted/master/data/the-counted-2016.csv'),  pd.read_csv('https://raw.githubusercontent.com/flother/thecounted/master/data/the-counted-2015.csv')] )


In [123]:
ny_by_police = by_police[by_police['state']=='NY']
depts = ['New York Police Department', 'New York City Police Department', 'New York State Police', 'New York court officer']
ny_by_police = ny_by_police[ny_by_police['lawenforcementagency'].isin(depts)]
not_nyc = ['Berne', 'Catskill', 'Mount Vernon']
ny_by_police = ny_by_police[~ny_by_police['city'].isin(not_nyc)]
ny_by_police[:1]


,uid,name,age,gender,raceethnicity,month,day,year,streetaddress,city,state,classification,lawenforcementagency,armed
239,2016461,Richard Gonzalez,55,Male,Hispanic/Latino,March,16,2016,416 E 137th St,New York,NY,Death in custody,New York Police Department,No


### Combine them

In [88]:
# combine them!
deaths = pd.concat([vehicle_deaths, crime_deaths], ignore_index=True)

In [89]:
deaths.isnull().sum()

date     0
kind     0
lat     74
lng     74
dtype: int64

In [91]:
deaths = deaths.dropna()
deaths.info()
# deaths.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6067 entries, 0 to 6140
Data columns (total 4 columns):
date    6067 non-null object
kind    6067 non-null object
lat     6067 non-null object
lng     6067 non-null object
dtypes: object(4)
memory usage: 237.0+ KB


In [71]:
deaths.columns = ['d', 'k', 'lat', 'lng' ]
deaths.to_json("deaths.json", orient="records")

In [ ]:
deaths.info

In [ ]:
deaths.lat.value_counts()